### Perform Imports

In [1]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
from keras import backend as K
import tensorflow as tf
from keras.layers import Conv2D, Dropout, LSTM, BatchNormalization, Input,Activation, MaxPool2D, Flatten, Dense,TimeDistributed
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from keras.layers.convolutional import ZeroPadding2D
from keras import metrics
import h5py
from sklearn.metrics import confusion_matrix
from utils import *
from data_utils import *
from models import *
import math
import gc

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
VIDEOS_DIR, IMAGES_DIR, classes, class_to_index, videos = get_global_variables()
print(VIDEOS_DIR)
print(IMAGES_DIR)
print(classes)
print(class_to_index)
print(videos)

../Videos/
../Images/
['Kicking', 'Riding-Horse', 'Running', 'SkateBoarding', 'Swing-Bench', 'Lifting', 'Swing-Side', 'Walking', 'Golf-Swing']
{'Lifting': 5, 'Swing-Bench': 4, 'Golf-Swing': 8, 'Riding-Horse': 1, 'SkateBoarding': 3, 'Kicking': 0, 'Running': 2, 'Swing-Side': 6, 'Walking': 7}
[['006', '017', '003', '016', '009', '013', '005', '012', '010', '015', '014', '007', '004', '011', '002', '001'], ['006', '003', '009', '005', '010', '007', '004', '002', '008', '001'], ['006', '009', '005', '010', '007', '004', '002', '008', '001'], ['006', '003', '009', '005', '010', '007', '004', '002', '008', '001'], ['006', '017', '003', '016', '009', '013', '005', '012', '010', '015', '014', '007', '004', '011', '002', '008', '001'], ['003', '005', '004', '002', '001'], ['006', '003', '009', '005', '010', '007', '004', '011', '002', '008', '001'], ['006', '017', '003', '016', '009', '013', '005', '012', '010', '015', '014', '007', '019', '004', '011', '018', '002', '008', '001'], ['006', '003'

In [3]:
K.tensorflow_backend._get_available_gpus()

[]

In [9]:
def end_to_end_gpu(input_shape):
    X_input = Input(input_shape)
    
    with tf.device('/cpu:0'):
        X = TimeDistributed(BatchNormalization(), name = 'BatchNorm_1')(X_input)
        X = TimeDistributed(Conv2D(32, (7, 7), strides = (4, 4), activation='relu', padding="same"), name="Conv_1a")(X)
        X = TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding="same") , name="Conv_1b")(X)
        X = TimeDistributed(MaxPool2D((2, 2)),  name = "Pool_1")(X)
        X = TimeDistributed(Dropout(0.2), name = "Dropout_1")(X)
    
    #with tf.device('/gpu:0'):     
        X = TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding = "same"), name ="Conv_2a")(X)
        X = TimeDistributed(MaxPool2D((2, 2)), name = "Pool_2")(X)
        X = TimeDistributed(Dropout(0.2), name= "Dropout_2")(X)

        X = TimeDistributed(Conv2D(32,(3,3), name='Conv_3a', activation='relu'))(X)
        X = TimeDistributed(MaxPool2D((2, 2), name = "Pool_3"))(X)
        X = TimeDistributed(Dropout(0.2), name = "Dropout_3")(X)

        X = TimeDistributed(Conv2D(8,(1,1), activation='relu'), name='Conv_1x1')(X)
        X = TimeDistributed(Flatten())(X)
        X = TimeDistributed(Dropout(0.3))(X)
        Y = TimeDistributed(Dense(9,activation='softmax',name='final'),name='Secondary_Output')(X)

        X = LSTM(48, return_sequences=False,dropout=0.3, name = "LSTM1")(X)
        #X = LSTM(32, return_sequences=False)(X)
        X = Dense(9, activation='softmax',name='Primary_Output')(X)

    return Model(X_input, outputs=[X, Y])


In [4]:
X_train, Y_train, X_test, Y_test = build_dataset_end_to_end((172, 172))

In [ ]:
X_train.shape, X_test.shape,Y_train.shape, Y_test.shape

In [ ]:
Y_train.shape

In [ ]:
x = np.random.randint(568)
plt.imshow(X_train[x, 0])
print(classes[Y_train[x]])

In [10]:
e2e = end_to_end_gpu((40, 172, 172, 3))

In [11]:
e2e.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 40, 172, 172, 0                                            
__________________________________________________________________________________________________
BatchNorm_1 (TimeDistributed)   (None, 40, 172, 172, 12          input_2[0][0]                    
__________________________________________________________________________________________________
Conv_1a (TimeDistributed)       (None, 40, 43, 43, 3 4736        BatchNorm_1[0][0]                
__________________________________________________________________________________________________
Conv_1b (TimeDistributed)       (None, 40, 43, 43, 3 9248        Conv_1a[0][0]                    
__________________________________________________________________________________________________
Pool_1 (Ti

In [12]:
e2e.compile(loss=['categorical_crossentropy', 'categorical_crossentropy'], loss_weights=[1, 0.6],
        metrics=['accuracy'],
        optimizer='adam')

In [5]:
X_train, Y_train = permute(X_train, Y_train)

In [6]:
X_train.shape

(647, 40, 172, 172, 3)

In [7]:
Y_train = convert_to_one_hot(Y_train, 9)

In [8]:
Y_train.shape

(647, 9)

In [9]:
Y_train2 = np.tile(Y_train, (40, 1, 1))

In [10]:
Y_train2 = Y_train2.transpose(1, 0, 2)


In [11]:
Y_train2.shape

(647, 40, 9)

### Data Generator

In [ ]:
size = X_train.shape[0]
batch_size = 80
suffix = 'aug'
for i in range(0,size,batch_size):
    if not os.path.exists('../Numpy/End2End/batch' + str(batch_size)):
        os.mkdir('../Numpy/End2End/batch' + str(batch_size))
    np.save('../Numpy/End2End/batch' + str(batch_size) + '/X_' + suffix + '_' + str(i) + '.npy',X_train[i:i+batch_size])
    np.save('../Numpy/End2End/batch' + str(batch_size) + '/Y_' + suffix + '_' + str(i) + '.npy',Y_train[i:i+batch_size])
    np.save('../Numpy/End2End/batch' + str(batch_size) + '/Y2_' + suffix + '_' + str(i) + '.npy',Y_train2[i:i+batch_size])

#### Generator Testing

In [ ]:
def generate_batch_data(train_size,batch_size,suffix = 'aug'):
    
    while 1:
        i = 0
        for i in range(0,train_size,batch_size):
            X_batch = '../Numpy/End2End/batch' + str(batch_size) + '/X_' + suffix + '_' + str(i) + '.npy'
            Y_batch =  '../Numpy/End2End/batch' + str(batch_size) + '/Y_' + suffix + '_' + str(i) + '.npy'
            Y_batch_2 = '../Numpy/End2End/batch' + str(batch_size) + '/Y2_' + suffix + '_' + str(i) + '.npy'
            
            batch =  (np.load(X_batch),[np.load(Y_batch),np.load(Y_batch_2)])
            yield batch

In [ ]:
batch_size = 80
train_size = X_train.shape[0]
steps = math.ceil(train_size/batch_size)
del X_train
del Y_train

In [ ]:
gen = generate_batch_data(train_size,batch_size)

In [ ]:
res = next(gen)
print(res[0].shape,res[1][0].shape)
e2e.train_on_batch(res[0],res[1])

In [ ]:
epochs = 5
for i in range(epochs):
    for j in range(steps):
        res = next(gen)
        print(res[0].shape,res[1][0].shape)
        e2e.train_on_batch(res[0],res[1])
        del res
    print("Epoch",i,"done..")    

## Training

In [39]:
for i in os.listdir():
    if i[-3:] == '.h5':
        mod = load_model(i)
        res = evaluate(mod,X_test,Y_test,verbose=False)
        print(i,res)

temp_11_11.h5 59.523809523809526
temp_13.h5 57.14285714285714
temp_11_1.h5 61.904761904761905
temp_10.h5 40.476190476190474
temp_8.h5 54.761904761904766
temp_11_6.h5 69.04761904761905
temp_11_14.h5 64.28571428571429
temp_11_12.h5 69.04761904761905
temp_14.h5 66.66666666666666
temp_11_2.h5 61.904761904761905
temp_11_13.h5 64.28571428571429
temp_11_3.h5 52.38095238095239
temp_11.h5 54.761904761904766
temp_11_4.h5 64.28571428571429
temp_7.h5 54.761904761904766
temp_11_10.h5 61.904761904761905
temp_12.h5 52.38095238095239
temp_11_5.h5 64.28571428571429
temp_11_0.h5 47.61904761904761
temp_9.h5 61.904761904761905


In [ ]:
for i in os.listdir('../models/End_End/'):
    print(i)
    if i[-3:] == '.h5':
        mod = load_model('../models/End_End/' + i)
        res = evaluate(mod,X_test_unseen,Y_test_unseen,verbose=False)
        print(i,res)

In [33]:
e2e = load_model('temp_11_6.h5')

In [34]:
evaluate(e2e,X_unseen,Y_unseen)

Max Preds time [0]
Pred Kicking Actual Kicking

Max Preds time [0]
Pred Kicking Actual Kicking

Max Preds time [0]
Pred Kicking Actual Kicking

Max Preds time [0]
Pred Kicking Actual Kicking

Max Preds time [2]
Pred Running Actual Riding-Horse

Max Preds time [1, 1, 1]
Pred Riding-Horse Actual Riding-Horse

Max Preds time [7, 7, 7, 7]
Pred Walking Actual Running

Max Preds time [2, 2, 2, 2]
Pred Running Actual SkateBoarding

Max Preds time [3, 3, 7, 8]
Pred SkateBoarding Actual SkateBoarding

Max Preds time [4, 4]
Pred Swing-Bench Actual Swing-Bench

Max Preds time [4, 4]
Pred Swing-Bench Actual Swing-Bench

Max Preds time [4, 4]
Pred Swing-Bench Actual Swing-Bench

Max Preds time [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
Pred Lifting Actual Lifting

Max Preds time [0]
Pred Kicking Actual Swing-Side

Max Preds time [6, 6, 6, 6, 6]
Pred Swing-Side Actual Swing-Side

Max Preds time [8, 8, 8, 8, 8, 8, 8, 7]
Pred Golf-Swing Actual Walking

Max Preds time [7, 7, 7, 7, 7]
Pred Walking Actual Walking

M

77.27272727272727

In [35]:
e2e.loss_weights

[1, 0.9]

In [36]:
e2e.loss_weights = [1,0.6]

In [37]:
#ten = TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=64, write_graph=True, write_grads=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=2, mode='auto')
epochs = 5
for i in range(epochs):
    e2e.fit(X_train, [Y_train, Y_train2], 
            epochs = 1, 
            batch_size = 64, 
            validation_split = 0.1,
            verbose=1)
    res = evaluate(e2e,X_test_unseen,Y_test_unseen,verbose=False)
    e2e.save('temp_11_' + str(i+10) + '.h5')
    print('temp_11_' + str(i+10) + '.h5',res)
    

Train on 582 samples, validate on 65 samples
Epoch 1/1
582/582 [==============================] - 311s 534ms/step - loss: 1.7581 - Primary_Output_loss: 0.7451 - Secondary_Output_loss: 1.1255 - Primary_Output_acc: 0.7337 - Secondary_Output_acc: 0.5703 - val_loss: 1.5054 - val_Primary_Output_loss: 0.6814 - val_Secondary_Output_loss: 0.9155 - val_Primary_Output_acc: 0.7077 - val_Secondary_Output_acc: 0.6900
temp_11_10.h5 63.63636363636363
Train on 582 samples, validate on 65 samples
Epoch 1/1
582/582 [==============================] - 307s 528ms/step - loss: 1.4523 - Primary_Output_loss: 0.5390 - Secondary_Output_loss: 1.0147 - Primary_Output_acc: 0.8144 - Secondary_Output_acc: 0.6223 - val_loss: 1.3077 - val_Primary_Output_loss: 0.5390 - val_Secondary_Output_loss: 0.8541 - val_Primary_Output_acc: 0.7692 - val_Secondary_Output_acc: 0.7481
temp_11_11.h5 77.27272727272727
Train on 582 samples, validate on 65 samples
Epoch 1/1
582/582 [==============================] - 292s 502ms/step - loss

## Testing

In [13]:
X_test_unseen, Y_test_unseen = build_test_dataset(image_size=(172, 172), stride = 10, max_len = 40, train = False)

In [14]:
X_test_unseen.shape, Y_test_unseen.shape

((22,), (22,))

In [ ]:
for i in range(9):
    mod = load_model('temp_'+ str(9) + '.h5')
    res = evaluate(mod,X_test_unseen,Y_test_unseen,verbose=False)
    print('temp_'+ str(i) + '.h5',res)
    break

In [ ]:
X_test_full, Y_test_full = build_test_dataset(image_size=(172, 172), stride = 10, max_len = 40, train = True)

In [ ]:
X_test_full.shape, Y_test_full.shape

In [ ]:
e2e = load_model('../models/End_End/temp0_ke_baad_3_epoch_phir_ulta_weights.h5')
print(evaluate(e2e, X_test_full, Y_test_full, verbose=False))

In [ ]:
print(res_norm,res_un)

In [ ]:
for i in os.listdir('../models/End_End/'):
    if i[-3:] == '.h5'and i[:5] == 'temp0':
        mod_16 = load_model('../models/End_End/' + i)
        res_un = evaluate(mod_16, X_test_unseen, Y_test_unseen,verbose=False)
        res_norm = evaluate(mod_16, X_test, Y_test, verbose=False)
        res_f = evaluate(mod_16, X_test_full, Y_test_full, verbose=False)
        print(i,res_un,res_norm,res_f)
        print(i, res_f)

In [ ]:
X_cropped, Y_cropped = build_image_dataset((172, 172))

In [ ]:
X_cropped = np.load('../Numpy/End2End/X_cropped.npy')
Y_cropped = np.load('../Numpy/End2End/Y_cropped.npy')

In [ ]:
[X_cropped[i].shape for i in range(len(X_cropped))]

In [ ]:
X_unpack = []
Y_unpack = []
for i in range(len(X_cropped)):
    for j in range(len(X_cropped[i])):
        X_unpack.append(X_cropped[i][j])
        Y_unpack.append(Y_cropped[i])

In [ ]:
X_unpack = np.array(X_unpack)

In [ ]:
X_unpack.shape

In [ ]:
Y_unpack = np.array(Y_unpack)
Y_unpack.shape

In [ ]:
for i in range(len(X_cropped)):
    print(i)
    plt.imshow(X_cropped[i][0][0].shape)
    plt.show()

In [ ]:
X_unpack, Y_unpack = permute(X_unpack, Y_unpack)

In [ ]:
X_unpack = X_unpack[:int(0.5*len(X_unpack))]
Y_unpack = Y_unpack[:int(0.5*len(Y_unpack))]

In [ ]:
np.save('../Numpy/End2End/X_unpack.npy', X_unpack)
np.save('../Numpy/End2End/Y_unpack.npy', Y_unpack)

In [ ]:
Y_unpack.shape